# 구글크롤링

In [ ]:
# 기존 chromium 삭제
!sudo apt remove chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


# Install selenium
pip install selenium

In [ ]:
!sudo apt update
!sudo apt install chromium

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [6]:
url = 'https://www.google.co.kr/'
driver.get(url)

In [14]:
search_box = driver.find_element(by=By.NAME, value='q')
search_box.send_keys('chromdriver')
search_box.send_keys(Keys.ENTER)
time.sleep(2)

In [16]:
divs = driver.find_elements(By.CSS_SELECTOR, '#search .g')
len(divs)

11

In [18]:
div = divs[0]
title = div.find_element(By.CSS_SELECTOR, '.LC20lb.MBeuO.DKV0Md').text
title

'WebDriver for Chrome - Downloads'

In [19]:
content = div.find_element(By.CSS_SELECTOR, '.VwiC3b.yXK7lf.MUxGbd').text.strip()
content

'If you are using Chrome version 113, please download ChromeDriver 113.0.5672.24.'

In [20]:
lines = []
for div in divs:
    try:
        title = div.find_element(By.CSS_SELECTOR, '.LC20lb.MBeuO.DKV0Md').text
        content = div.find_element(By.CSS_SELECTOR, '.VwiC3b.yXK7lf.MUxGbd').text
    except:
        continue
    lines.append([title, content])

In [21]:
import pandas as pd
df = pd.DataFrame(lines, columns=['title', 'content'])
df

,title,content
0,WebDriver for Chrome - Downloads,"If you are using Chrome version 113, please do..."
1,Install browser drivers,"Mar 2, 2023 — Through WebDriver, Selenium supp..."
2,ChromeDriver · SeleniumHQ/selenium Wiki,"Dec 26, 2021 — A browser automation framework ..."
3,How to run Selenium tests on Chrome using Chro...,"Mar 20, 2023 — To execute tests in the Chrome ..."
4,Selenium.WebDriver.ChromeDriver 112.0.5615.4900,"Install Chrome Driver (Win32, macOS, macOS arm..."
5,undetected-chromedriver,'Selenium.webdriver.Chrome replacement with co...
6,ChromeDriver not available for chrome version ...,to ChromeDriver Users. i recently updated my g...
